## import libraries

In [1]:
import cv2
import numpy as np
import time
import mediapipe as mp
import math
import serial
from matplotlib import pyplot as plt

## Use mediapipe library to detect hand and return some positions

In [2]:
class handDetector():
    def __init__(self, mode=False):
        self.mode = mode
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5 
        )
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]


    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,self.mpHands.HAND_CONNECTIONS)
        return img


    def findPosition(self, img, handNo=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            if handNo < len(self.results.multi_hand_landmarks):
                hand = self.results.multi_hand_landmarks[handNo]
                h, w, _ = img.shape
                for id, lm in enumerate(hand.landmark):
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmList.append((id, cx, cy))
        return lmList

## here we preprocess webcam and serial transfare details

In [ ]:
detector = handDetector()
wCam, hCam = 640,480
cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
pTime = 100
if not cap.isOpened():
    print("Error: Cannot access the webcam")
    exit()
frame_width = wCam
frame_height = hCam
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame_width, frame_height))
pTime = 0
distance = 0
RGBchanel = 0
arduino = serial.Serial('COM5',9600,timeout=1)    #write your COM num port

## Here we write conditions (that which gestures do what) and webcam opening
### we use openCV and lmList to find distance and RGB chanel number

In [ ]:
while True:
    ret, frame = cap.read()
    frame = detector.findHands(frame)
    lmList = detector.findPosition(frame,draw=False)
    if len(lmList) != 0:
        x1,y1 = lmList[4][1],lmList[4][2]
        x2,y2 = lmList[8][1],lmList[8][2]

        cx,cy = (x1+x2)//2,(y1+y2)//2

        cv2.circle(frame,(x1,y1),4,(0,255,0),cv2.FILLED)
        cv2.circle(frame,(x2,y2),4,(0,255,0),cv2.FILLED)
        cv2.line(frame,(x1,y1),(x2,y2),(20,224,255),3)

        fing_3 = lmList[10][2] < lmList[11][2] and lmList[11][2] < lmList[12][2]
        fing_4 = lmList[14][2] < lmList[15][2] and lmList[15][2] < lmList[16][2]
        fing_5 = lmList[18][2] < lmList[19][2] and lmList[19][2] < lmList[20][2]

        distance = math.hypot(x2-x1,y2-y1)
        if distance<25:
            cv2.circle(frame,(cx,cy),5,(20,86,255),3)

        if fing_3 and fing_4 and fing_5:
            RGBchannel = 1
        if not(fing_3) and not(fing_4) and not(fing_5):
            RGBchannel = 3
        if not(fing_3) and fing_4 and fing_5:
            RGBchannel = 1
        if fing_3 and not(fing_4) and fing_5:
            RGBchannel = 1
        if fing_3 and fing_4 and not(fing_5):
            RGBchannel = 1
        if not(fing_3) and not(fing_4) and fing_5:
            RGBchannel = 2
        if not(fing_3) and fing_4 and not(fing_5):
            RGBchannel = 2
        if fing_3 and not(fing_4) and not(fing_5):
            RGBchannel = 2
            
    if not ret:
        print("Failed to grab frame")
        break

    out.write(frame)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(frame,f'FPS:{int(fps)}',(40,70),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255),3)
    cv2.imshow('Webcam', frame)
    
    distance = distance/20
    if distance > 9 :
        distance = 9

    RGBchanel = str(RGBchannel)
    int2strDistance = str(int(distance))

    arduino.write(str(RGBchannel+"#"+int2strDistance).encode())

    cv2.waitKey(1)
    cv2.imshow('Webcam', frame)
    cv2.waitKey(1)

cap.release()
out.release()
cv2.destroyAllWindows()


0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
0#0
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#5
3#5
3#5
3#5
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#4
3#5
3#4
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#5
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#6
3#7
3#6
3#6
3#6
3#6
3#7
3#7
3#7
3#7
3#7
3#8
3#8
3#8
3#8
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
3#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9
2#9


KeyboardInterrupt: 